## Part 1: Preprocessing

In [2]:
# Import our dependencies
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
import pandas as pd
import numpy as np
from tensorflow.keras.models import Model
from tensorflow.keras import layers

#  Import and read the attrition data
attrition_df = pd.read_csv('https://static.bc-edx.com/ai/ail-v-1-0/m19/lms/datasets/attrition.csv')
attrition_df.head()

,Age,Attrition,BusinessTravel,Department,DistanceFromHome,Education,EducationField,EnvironmentSatisfaction,HourlyRate,JobInvolvement,...,PerformanceRating,RelationshipSatisfaction,StockOptionLevel,TotalWorkingYears,TrainingTimesLastYear,WorkLifeBalance,YearsAtCompany,YearsInCurrentRole,YearsSinceLastPromotion,YearsWithCurrManager
0,41,Yes,Travel_Rarely,Sales,1,2,Life Sciences,2,94,3,...,3,1,0,8,0,1,6,4,0,5
1,49,No,Travel_Frequently,Research & Development,8,1,Life Sciences,3,61,2,...,4,4,1,10,3,3,10,7,1,7
2,37,Yes,Travel_Rarely,Research & Development,2,2,Other,4,92,2,...,3,2,0,7,3,3,0,0,0,0
3,33,No,Travel_Frequently,Research & Development,3,4,Life Sciences,4,56,3,...,3,3,0,8,3,3,8,7,3,0
4,27,No,Travel_Rarely,Research & Development,2,1,Medical,1,40,3,...,3,4,1,6,3,3,2,2,2,2


In [3]:
# Determine the number of unique values in each column.
attrition_df.nunique()

Age                         43
Attrition                    2
BusinessTravel               3
Department                   3
DistanceFromHome            29
Education                    5
EducationField               6
EnvironmentSatisfaction      4
HourlyRate                  71
JobInvolvement               4
JobLevel                     5
JobRole                      9
JobSatisfaction              4
MaritalStatus                3
NumCompaniesWorked          10
OverTime                     2
PercentSalaryHike           15
PerformanceRating            2
RelationshipSatisfaction     4
StockOptionLevel             4
TotalWorkingYears           40
TrainingTimesLastYear        7
WorkLifeBalance              4
YearsAtCompany              37
YearsInCurrentRole          19
YearsSinceLastPromotion     16
YearsWithCurrManager        18
dtype: int64

In [4]:
# Create y_df with the Attrition and Department columns
y_df = attrition_df.loc[:, ['Attrition', 'Department']]

In [5]:
# Create a list of at least 10 column names to use as X data
columns_X = ['Education', 'Age', 'DistanceFromHome', 'JobSatisfaction', 'OverTime',
             'StockOptionLevel', 'WorkLifeBalance', 'YearsAtCompany',
             'YearsSinceLastPromotion', 'NumCompaniesWorked']

# Create X_df using your selected columns
X_df = attrition_df[columns_X]

# Show the data types for X_df
print(X_df.dtypes)

Education                   int64
Age                         int64
DistanceFromHome            int64
JobSatisfaction             int64
OverTime                   object
StockOptionLevel            int64
WorkLifeBalance             int64
YearsAtCompany              int64
YearsSinceLastPromotion     int64
NumCompaniesWorked          int64
dtype: object


In [6]:
# Split the data into training and testing sets
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(X_df, y_df, test_size=0.2, random_state=42)

In [7]:
# Convert your X data to numeric data types however you see fit
# Add new code cells as necessary
# The only column with non-numeric data is the OverTime column.  This can be converted with simple lambda function.
X_train['OverTime'] = X_train['OverTime'].apply(lambda x: 1 if x == 'Yes' else 0)
X_test['OverTime'] = X_test['OverTime'].apply(lambda x: 1 if x == 'Yes' else 0)


In [8]:
# Create a StandardScaler
scaler = StandardScaler()

# Fit the StandardScaler to the training data
scaler.fit(X_train)

# Scale the training and testing data
X_train_scaled = scaler.transform(X_train)
X_test_scaled = scaler.transform(X_test)


In [9]:
from sklearn.preprocessing import OneHotEncoder
# Create a OneHotEncoder for the Department column
encoder_dep = OneHotEncoder(sparse_output=False)

# Fit the encoder to the training data
encoder_dep.fit(y_train[['Department']])

# Create two new variables by applying the encoder
# to the training and testing data
y_train_department_encoded = encoder_dep.transform(y_train[['Department']])
y_test_department_encoded = encoder_dep.transform(y_test[['Department']])

In [10]:
# Create a OneHotEncoder for the Attrition column
encoder_att = OneHotEncoder(sparse_output=False)

# Fit the encoder to the training data
encoder_att.fit(y_train[['Attrition']])

# Create two new variables by applying the encoder
# to the training and testing data
y_train_attrition_encoded = encoder_att.transform(y_train[['Attrition']])
y_test_attrition_encoded = encoder_att.transform(y_test[['Attrition']])


## Create, Compile, and Train the Model

In [11]:
from tensorflow.keras.layers import Input, Dense
from tensorflow.keras.optimizers import Adam
# Find the number of columns in the X training data
num_columns = X_train.shape[1]

# Create the input layer
input_layer = Input(shape=(num_columns,), name='input')

# Create at least two shared layers
shared_layer_1 = Dense(64, activation='relu', name='shared1')(input_layer)
shared_layer_2 = Dense(128, activation='relu', name='shared2')(shared_layer_1)

In [12]:
# Create a branch for Department
# with a hidden layer and an output layer
department_hidden = Dense(32, activation='relu', name='department_hidden')(shared_layer_2)
department_output = Dense(3, activation='softmax', name='department_output')(department_hidden)

In [13]:
# Create a branch for Attrition
# with a hidden layer and an output layer
attrition_hidden = Dense(32, activation='relu', name='attrition_hidden')(shared_layer_2)
attrition_output = Dense(2, activation='softmax', name='attrition_output')(attrition_hidden)

In [14]:
# Create the model
model = Model(inputs=input_layer, outputs=[department_output, attrition_output])

# Compile the model
model.compile(optimizer=Adam(learning_rate=0.001),
              loss={'department_output': 'categorical_crossentropy',
                    'attrition_output': 'categorical_crossentropy'},
              metrics={'department_output': 'accuracy',
                       'attrition_output': 'accuracy'})

# Summarize the model
model.summary()

Model: "model"
__________________________________________________________________________________________________
 Layer (type)                Output Shape                 Param #   Connected to                  
 input (InputLayer)          [(None, 10)]                 0         []                            
                                                                                                  
 shared1 (Dense)             (None, 64)                   704       ['input[0][0]']               
                                                                                                  
 shared2 (Dense)             (None, 128)                  8320      ['shared1[0][0]']             
                                                                                                  
 department_hidden (Dense)   (None, 32)                   4128      ['shared2[0][0]']             
                                                                                              

In [15]:
# Train the model
history = model.fit(
    X_train_scaled,
    {'department_output': y_train_department_encoded, 'attrition_output': y_train_attrition_encoded},
    validation_data=(X_test_scaled, {'department_output': y_test_department_encoded, 'attrition_output': y_test_attrition_encoded}),
    epochs=100,
    batch_size=32,
    verbose=1
)

Epoch 1/100
37/37 [==============================] - 1s 5ms/step - loss: 1.3203 - department_output_loss: 0.8461 - attrition_output_loss: 0.4743 - department_output_accuracy: 0.6344 - attrition_output_accuracy: 0.8240 - val_loss: 1.1944 - val_department_output_loss: 0.8013 - val_attrition_output_loss: 0.3931 - val_department_output_accuracy: 0.6667 - val_attrition_output_accuracy: 0.8673
Epoch 2/100
37/37 [==============================] - 0s 1ms/step - loss: 1.1780 - department_output_loss: 0.7771 - attrition_output_loss: 0.4008 - department_output_accuracy: 0.6497 - attrition_output_accuracy: 0.8393 - val_loss: 1.1894 - val_department_output_loss: 0.7992 - val_attrition_output_loss: 0.3902 - val_department_output_accuracy: 0.6667 - val_attrition_output_accuracy: 0.8469
Epoch 3/100
37/37 [==============================] - 0s 1ms/step - loss: 1.1439 - department_output_loss: 0.7666 - attrition_output_loss: 0.3773 - department_output_accuracy: 0.6505 - attrition_output_accuracy: 0.8529 

In [16]:
# Evaluate the model with the testing data
# Evaluate the model on the test data
evaluation_results = model.evaluate(
    X_test_scaled,
    {'department_output': y_test_department_encoded, 'attrition_output': y_test_attrition_encoded},
    verbose=1
)



10/10 [==============================] - 0s 722us/step - loss: 4.9298 - department_output_loss: 2.7660 - attrition_output_loss: 2.1638 - department_output_accuracy: 0.5170 - attrition_output_accuracy: 0.8061


In [17]:
# Print the accuracy for both department and attrition
# Print the evaluation results
print(f"Department preditions accuracy: {evaluation_results[3]}")
print(f"Attrition predictions accuracy: {evaluation_results[4]}")

Department preditions accuracy: 0.5170068144798279
Attrition predictions accuracy: 0.8061224222183228


# Summary

In the provided space below, briefly answer the following questions.

1. Is accuracy the best metric to use on this data? Why or why not?

2. What activation functions did you choose for your output layers, and why?

3. Can you name a few ways that this model might be improved?

YOUR ANSWERS HERE

1. Accuracy may not be the best metric for this data, especially if the classes are imbalanced (e.g., more instances of one class than another). In such cases, accuracy can be misleading because it doesn't account for the distribution of classes and might not reflect the model's true performance. Metrics like precision, recall, or F1-score are often more informative in such scenarios.
2. For the department_output layer, I chose softmax activation because it's suitable for multi-class classification tasks. Softmax outputs probabilities that sum to 1 across all classes.
For the attrition_output layer, since attrition is binary, sigmoid activation is suitable. Sigmoid outputs a probability score between 0 and 1, which can be interpreted as the likelihood of an employee leaving.
3. Consider using metrics like precision, recall, F1-score, or ROC-AUC for evaluating model performance, especially given the binary nature of the attrition prediction.
Explore techniques specific to handling imbalanced classes, such as class weighting, oversampling, or using evaluation metrics that are robust to class imbalance.
Implement regularization techniques like dropout or L2 regularization to prevent overfitting, especially if the dataset is small or noisy.
Experiment with different model architectures, hyperparameters, and feature engineering techniques to improve predictive accuracy and robustness.